In [1]:
import json
studentsJson = [json.loads(line) for line in open('students.json','r')]
print(studentsJson)

[{'_id': 0, 'name': 'aimee Zank', 'scores': [{'score': 1.463179736705023, 'type': 'exam'}, {'score': 11.78273309957772, 'type': 'quiz'}, {'score': 35.8740349954354, 'type': 'homework'}]}, {'_id': 1, 'name': 'Aurelia Menendez', 'scores': [{'score': 60.06045071030959, 'type': 'exam'}, {'score': 52.79790691903873, 'type': 'quiz'}, {'score': 71.76133439165544, 'type': 'homework'}]}, {'_id': 2, 'name': 'Corliss Zuk', 'scores': [{'score': 67.03077096065002, 'type': 'exam'}, {'score': 6.301851677835235, 'type': 'quiz'}, {'score': 66.28344683278382, 'type': 'homework'}]}, {'_id': 3, 'name': 'Bao Ziglar', 'scores': [{'score': 71.64343899778332, 'type': 'exam'}, {'score': 24.80221293650313, 'type': 'quiz'}, {'score': 42.26147058804812, 'type': 'homework'}]}, {'_id': 4, 'name': 'Zachary Langlais', 'scores': [{'score': 78.68385091304332, 'type': 'exam'}, {'score': 90.2963101368042, 'type': 'quiz'}, {'score': 34.41620148042529, 'type': 'homework'}]}, {'_id': 5, 'name': 'Wilburn Spiess', 'scores': [

In [2]:
!pip install pymongo
!pip install dnspython
!pip install pymongo[srv]
from pymongo import MongoClient

In [3]:
client = MongoClient("localhost:27017")
db = client['student']
collection = db['student_details']

In [4]:
db.collection.insert_many(studentsJson)

In [5]:
# converting json data in to a dataframe
import pandas as pd
df=pd.DataFrame(data=studentsJson)
df.head()

,_id,name,scores
0,0,aimee Zank,"[{'score': 1.463179736705023, 'type': 'exam'},..."
1,1,Aurelia Menendez,"[{'score': 60.06045071030959, 'type': 'exam'},..."
2,2,Corliss Zuk,"[{'score': 67.03077096065002, 'type': 'exam'},..."
3,3,Bao Ziglar,"[{'score': 71.64343899778332, 'type': 'exam'},..."
4,4,Zachary Langlais,"[{'score': 78.68385091304332, 'type': 'exam'},..."


In [6]:
! pip install pymongo
import pymongo
client = pymongo.MongoClient("mongodb://localhost:27017/")
db1=client['studentdb']
coll=db1['student_details']

In [7]:
name=[]
exam_score=[]
quiz_score=[]
homework_score=[]
for i in range(0,len(df)):
    name.append(df['name'][i])
    exam_score.append(df['scores'][i][0]['score'])
    quiz_score.append(df['scores'][i][1]['score'])
    homework_score.append(df['scores'][i][2]['score'])
new=zip(name,exam_score,quiz_score,homework_score)
new_df=pd.DataFrame(new,columns=['name','exam_score','quiz_score','homework_score'])
new_df   

,name,exam_score,quiz_score,homework_score
0,aimee Zank,1.463180,11.782733,35.874035
1,Aurelia Menendez,60.060451,52.797907,71.761334
2,Corliss Zuk,67.030771,6.301852,66.283447
3,Bao Ziglar,71.643439,24.802213,42.261471
4,Zachary Langlais,78.683851,90.296310,34.416201
...,...,...,...,...
195,Linnie Weigel,52.445784,90.777505,11.750084
196,Santiago Dollins,52.040526,33.633001,78.792574
197,Tonisha Games,38.512696,31.162876,79.158564
198,Timothy Harrod,11.907567,20.518800,64.856504


In [8]:
#1 Find the student name who scored maximum scores in all (exam, quiz and homework)?

print("\nMaximum Mark in Exam Type is :\n")
exam_max = coll.find_one( sort = [ ( "exam_score" , -1 ) ] )
print(exam_max["name"])

print("\nMaximum Mark in Quiz Type is :\n")
quiz_max = coll.find_one( sort = [ ( 'quiz_score' , -1 ) ] )
print(quiz_max["name"])

print("\nMaximum Mark in Homework Type is :\n")
homework_max = coll.find_one( sort = [ ( 'homework_score' , -1 ) ] )
print(homework_max["name"])


Maximum Mark in Exam Type is :



TypeError: 'NoneType' object is not subscriptable

In [9]:
#saving dataframe to mongodb
def importdata(data):
    data.reset_index(drop=True, inplace=True)
    data_dict = data.to_dict("records")
    return data_dict
db=client['studentdb']
stu_coll=db['student_details']
stu_coll.insert_many(importdata(new_df))

In [10]:
#2)Find students who scored below average in the exam and pass mark is 40%?
avg=new_df['exam_score'].mean()
passmarks=40
lst=[]
for i in coll.find({"$and":[{"exam_score":{"$lt":avg}},{"exam_score":{"$gt":passmarks}}]}):
    lst.append(i['name'])
df_lst=pd.DataFrame(lst) 
df_lst

,0
0,Wilburn Spiess
1,Denisha Cast
2,Gisela Levin
3,Tressa Schwing
4,Rosana Vales
5,Tamika Schildgen
6,Gwyneth Garling
7,Kayce Kenyon
8,Terica Brugger
9,Chad Rahe


In [11]:
# 4)Find students who scored below pass mark and assigned them as fail, and above pass mark as pass in all the categories.
import numpy as np
df1=new_df
passmark=40
lst1=[]
lst2=[]
lst3=[]
lst1 = np.where((df1['exam_score'] > passmark), "Pass", "Fail")
df1.insert(loc = 2,
          column = 'exam_res',
          value = lst1)

lst2 = np.where((df1['quiz_score'] > passmark), "Pass", "Fail")
df1.insert(loc = 4,
          column = 'quiz_res',
          value = lst2)

lst3 = np.where((df1['homework_score'] > passmark), "Pass", "Fail")
df1.insert(loc = 6,
          column = 'homework_res',
          value = lst3)

df1

,name,exam_score,exam_res,quiz_score,quiz_res,homework_score,homework_res
0,aimee Zank,1.463180,Fail,11.782733,Fail,35.874035,Fail
1,Aurelia Menendez,60.060451,Pass,52.797907,Pass,71.761334,Pass
2,Corliss Zuk,67.030771,Pass,6.301852,Fail,66.283447,Pass
3,Bao Ziglar,71.643439,Pass,24.802213,Fail,42.261471,Pass
4,Zachary Langlais,78.683851,Pass,90.296310,Pass,34.416201,Fail
...,...,...,...,...,...,...,...
195,Linnie Weigel,52.445784,Pass,90.777505,Pass,11.750084,Fail
196,Santiago Dollins,52.040526,Pass,33.633001,Fail,78.792574,Pass
197,Tonisha Games,38.512696,Fail,31.162876,Fail,79.158564,Pass
198,Timothy Harrod,11.907567,Fail,20.518800,Fail,64.856504,Pass


In [12]:
df1

,name,exam_score,exam_res,quiz_score,quiz_res,homework_score,homework_res
0,aimee Zank,1.463180,Fail,11.782733,Fail,35.874035,Fail
1,Aurelia Menendez,60.060451,Pass,52.797907,Pass,71.761334,Pass
2,Corliss Zuk,67.030771,Pass,6.301852,Fail,66.283447,Pass
3,Bao Ziglar,71.643439,Pass,24.802213,Fail,42.261471,Pass
4,Zachary Langlais,78.683851,Pass,90.296310,Pass,34.416201,Fail
...,...,...,...,...,...,...,...
195,Linnie Weigel,52.445784,Pass,90.777505,Pass,11.750084,Fail
196,Santiago Dollins,52.040526,Pass,33.633001,Fail,78.792574,Pass
197,Tonisha Games,38.512696,Fail,31.162876,Fail,79.158564,Pass
198,Timothy Harrod,11.907567,Fail,20.518800,Fail,64.856504,Pass


In [14]:
# Find the total and average of the exam, quiz and homework and store them in a separate collection.
df1['total'] = (df1['exam_score']+df1['quiz_score']+df1['homework_score'])
df1['mean'] = ((df1['exam_score']+df1['quiz_score']+df1['homework_score'])/3)

df2=df1

In [17]:
col_4 = db["total_mean"]
res = coll["total_mean"]
res.insert_many(importdata(df2))

In [18]:
print("Total and Average of The Exam, Quiz and Homework \nStore Them in A Aeparate Collection\n")
for i in res.find({},{'_id':0,"name":1,"total":1,"mean":1}):
    print(i)


Total and Average of The Exam, Quiz and Homework 
Store Them in A Aeparate Collection

{'name': 'aimee Zank', 'total': 49.11994783171814, 'mean': 16.373315943906046}
{'name': 'Aurelia Menendez', 'total': 184.61969202100374, 'mean': 61.53989734033458}
{'name': 'Corliss Zuk', 'total': 139.61606947126907, 'mean': 46.53868982375636}
{'name': 'Bao Ziglar', 'total': 138.70712252233457, 'mean': 46.23570750744486}
{'name': 'Zachary Langlais', 'total': 203.3963625302728, 'mean': 67.79878751009093}
{'name': 'Wilburn Spiess', 'total': 134.0186975547794, 'mean': 44.67289918492647}
{'name': 'Jenette Flanders', 'total': 147.2203575476617, 'mean': 49.07345251588723}
{'name': 'Salena Olmos', 'total': 229.3959334774732, 'mean': 76.46531115915774}
{'name': 'Daphne Zheng', 'total': 112.71677331754347, 'mean': 37.57225777251449}
{'name': 'Sanda Ryba', 'total': 220.08328118626565, 'mean': 73.36109372875522}
{'name': 'Denisha Cast', 'total': 199.8843572885121, 'mean': 66.62811909617069}
{'name': 'Marcus Blo